# Numerical optimization
- a quick primer on numerical optimization (NO) for neural networks
- key to understanding NO for networks: networks are just functions, general NO methods apply
- let's review some that have been applied in NNs on 3d functions
- we look at the following methods: stochastic gradient descent, momentum method, rmsprop, adagrad, adadelta, nesterov accelerated gradient
- also some specialized strategies are applied: learning rate decay schedule
- a nice [review](http://www.matthewzeiler.com/pubs/googleTR2012/googleTR2012.pdf) of some functions (but biased of course)
- some nice test functions are taken from [here](https://en.wikipedia.org/wiki/Test_functions_for_optimization)
- we are using methods from the package [optim](https://github.com/torch/optim)

In [15]:
Plot = require 'itorch.Plot'
require 'optim'
require 'utils'
require 'gnuplotx'

In [2]:
-- this is a 'training' function that starts at a given point and then runs
-- the optimizer 'optim_func' for a 'nsteps' steps
function compute_track(x_init, f, df, optim_func, opt_config, nsteps)
    
    -- number of parameters
    local n = x_init:nElement()
    
    -- start at given initial point
    local x = x_init:clone()
    
    -- allocate nsteps+1 spaces for optimizer path (+1 is for initial state)
    local track = torch.zeros(nsteps+1,n+1)
    track[{1,{1,n}}] = x
    track[{1,n+1}] = f(x)
    
    -- opt_state carries information optimizer wants to remember between optimization steps
    local opt_state = {}

    for i=2,nsteps+1 do
        -- compute the next x values according to the optimizer
        x = optim_func(function (x) return f(x), df(x) end, x, opt_config, opt_state)
        track[{i,{1,n}}] = x

        -- store the new point in the track array
        local fx = f(x)
        -- we need the path to be visible, so we lift it 10% above the error surface
        track[{i,n+1}] = fx + 0.05 * math.abs(fx)
    end
    
    return track
end

# 1D function optimization

In [3]:
?torch.pow

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++	
[res] torch.pow([res,] x, n)


Let x be a tensor and n a number.

 y=torch.pow(x,n) returns a new tensor with the elements of x to the 
power of n .

 y=torch.pow(n,x) returns, a new tensor with n to the power of the 
elements of x .

 x:pow(n) replaces all elements in-place with the elements of x to 
the power of n .	
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++	



In [4]:
-- work with 1D functions first

pb_f = function (x) return x*x end
pb_df = function (x) return x*2 end
pb_range = {-5, 5, 0, 25}

In [5]:
xs = torch.linspace(-5,5,40)
ys = utils.map(pb_f, xs)

In [6]:
x_init = torch.Tensor{4}
nsteps = 10
sgd_cfg = { learningRate = 0.01 }
sgd_track = compute_track(x_init, pb_f, pb_df, optim.sgd, sgd_cfg, nsteps)

In [7]:
p = Plot()
p:line(xs,ys)
--circle(sgd_track[{{},1}],sgd_track[{{},2}],'red')
--p:xaxis('x'):yaxis('err'):title('Optimizer path')
p:draw()

In [8]:
-- we will start with a simple test function - a 2D quadratic

-- the function value
sphere_f = function (x) return torch.sum(torch.pow(x,2)) end
-- the function gradient
sphere_df = function (x) return x * 2 end
sphere_range = {-5, 5, -5, 5, 0, 30}

-- Beale's function
beale_f = function (x) return (1.5 - x[1] + x[1]*x[2])^2 + (2.25 - x[1] + x[1]*x[2]^2)^2 + (2.625 - x[1] + x[1]*x[2]^3)^2 end
beale_df = function (x)
    local df_dx = torch.zeros(2)
    local a = 1.5 - x[1] + x[1]*x[2]
    local b = 2.25 - x[1] + x[1]*x[2]^2
    local c = 2.625 - x[1] + x[1]*x[2]^3
    
    df_dx[1] = 2*a*(-1 + x[2]) + 2*b*(-1+x[2]^2) + 2*c*(-1 + x[2]^3)
    df_dx[2] = 2*a*x[1] + 4*b*x[1]*x[2] + 6*c*x[1]*x[2]^2
    return df_dx
end
beale_range = {-5, 5, -5, 5, 0, 200}

-- Golstein-Price function
gp_f = function (x) return (1+(x[1]+x[2]+1)^2*(19-14*x[1]+3*x[1]^2-14*x[2]+6*x[1]*x[2]+3*x[2]^2))*(30+(2*x[1]-3*x[2])^2*(18-32*x[1]+12*x[1]^2+48*x[2]-36*x[1]*x[2]+27*x[2]^2)) end
gp_df = function (x)
    local a = x[1]+x[2]+1
    local b = 19-14*x[1]+3*x[1]^2-14*x[2]+6*x[1]*x[2]+3*x[2]^2
    local c = 2*x[1]-3*x[2]
    local d = 18-32*x[1]+12*x[1]^2+48*x[2]-36*x[1]*x[2]+27*x[2]^2
    local df_dx = torch.zeros(2)
    df_dx[1] = (1 + a^2*b)*(4*c*d - c^2*(32+36*x[2]-24*x[1])) + (30+c^2*d)*(2*a*b + a^2*(-14+6*x[1]+6*x[2]))
    df_dx[2] = (1 + a^2*b)*(c^2*(48-36*x[1]+54*x[2])-6*c*d) + (30+c^2*d)*(2*a*b + a^2*(6*x[1]+6*x[2]-14))
    return df_dx
end
gp_range = {-2, 2, -2, 2, 0, 1.2e6}

-- Booth's function
booth_f = function (x) return (x[1] + 2*x[2]-7)^2 + (2*x[1] + x[2] - 5)^2 end
booth_df = function (x)
    local df_dx = torch.zeros(2)
    local a = x[1] + 2*x[2]-7
    local b = 2*x[1] + x[2] - 5
    df_dx[1] = 2*a + 4*b
    df_dx[2] = 4*a + 2*b
    return df_dx
end
--booth_range = {-11, 11, -11, 11, 0, 2500}
booth_range = {0, 11, 0, 11, 0, 2500}

-- Styblinski-Tang function
st_f = function (x) return 0.5 * (x[1]^4 + x[2]^4 - 16*(x[1]^2 + x[2]^2) + 5*(x[1] + x[2])) end
st_df = function (x)
    local df_dx = torch.zeros(2)
    df_dx[1] = 0.5 * (4*x[1]^3 - 32*x[1] + 5)
    df_dx[2] = 0.5 * (4*x[2]^3 - 32*x[2] + 5)
    return df_dx
end
--st_range = {-5, 5, -5, 5, -100, 200}
st_range = {0, 5, 0, 5, -100, 200}


-- Revisiting red-vs-blue
N = 100
stdev = 1.5
Reds, Blues = unpack(utils.make_data(N, stdev))
train_data = torch.cat(Reds, Blues, 1)
train_labels = torch.cat(torch.ones(N/2), - torch.ones(N/2), 1)

-- the dataset_error function renamed and simplified to only depend on ps
function rb_f(ps)
  local tloss = 0.
  local dw1 = 0
  local dw2 = 0
    
  for i=1,train_data:size(1) do
    -- simply add all losses per data rule
    local xs = train_data[i]
    local y = math.tanh(xs[1] * ps[1] + xs[2] * ps[2])
    local target = train_labels[i]
    -- we compute the loss
    tloss = tloss + 0.5 * (y-target)^2
  end
  return tloss / train_data:size(1)
end

-- computes the gradient of the error function
function rb_df(ps)
  local dw1 = 0
  local dw2 = 0
    
  for i=1,train_data:size(1) do
    -- simply add all losses per data rule
    local xs = train_data[i]
    local y = math.tanh(xs[1] * ps[1] + xs[2] * ps[2])
    local target = train_labels[i]
        
    -- and we compute the gradient at the same time
    dw1 = dw1 + (y - target) * (1 - y^2) * xs[1]
    dw2 = dw2 + (y - target) * (1 - y^2) * xs[2]
  end
  return torch.Tensor{dw1,dw2} / train_data:size(1)
end

rb_range = {-1, 1, -1, 1, 0, 2}

In [9]:
f, df, range = sphere_f, sphere_df, sphere_range

In [10]:
-- quick sanity check of the computed gradient
-- the numbers should approximately match between elements
x = torch.zeros(2)
x:uniform(-2,2)
print('Evaluation point')
print(x)
print('Analytically computed gradient')
print(df(x))

print('Numerically estimated gradient')
df_num = torch.Tensor{(f(x + torch.Tensor{0.00001, 0}) - f(x))/0.00001,
                      (f(x + torch.Tensor{0, 0.00001}) - f(x))/0.00001}
print(df_num)

print('Difference')
print(df(x) - df_num)

Evaluation point	
 0.1001
 1.1021
[torch.DoubleTensor of size 2]

Analytically computed gradient	
 0.2002
 2.2041
[torch.DoubleTensor of size 2]

Numerically estimated gradient	
 0.2002
 2.2041
[torch.DoubleTensor of size 2]

Difference	
1e-06 *
-10.0000
-10.0000
[torch.DoubleTensor of size 2]



In [11]:
-- utility functions for plotting the error surface
function meshgrid(x,y)
    local xx = torch.repeatTensor(x, y:size(1),1)
    local yy = torch.repeatTensor(y:view(-1,1), 1, x:size(1))
    return xx, yy
end

function apply_by_element(x, y, f)
    z = torch.zeros(x:size())
    xij = torch.zeros(2)
    for i=1,x:size(1) do
        for j=1,x:size(2) do
            z[i][j] = f(torch.Tensor{x[i][j],y[i][j]})
        end
    end
    return z
end

In [12]:
-- function domain
x = torch.linspace(range[1], range[2], 40)
y = torch.linspace(range[3], range[4], 40)
xx, yy = meshgrid(x, y)
zz = apply_by_element(xx, yy, f)

In [16]:
gnuplotx.figure(1)
gnuplotx.title('Optimized function')
gnuplotx.splot(xx,yy,zz)

In [14]:
-- setup initial conditions for our function
x_init = torch.Tensor{3, 3}

-- this is the number of steps the optimizers will run
nsteps = 40

In [ ]:
sgd_config = {
    learningRate = 0.1,
    learningRateDecay = 1e-2,
    momentum = 0,
    dampening = 0,
}

sgdm_config = {
    learningRate = 0.1,
    learningRateDecay = 1e-2,
    momentum = 0.9,
    dampening = 0,
}

adadelta_config = {
    rho = 0.9,
    eps = 1e-6
}

adagrad_config = {
    learningRate = 0.01,
    learningRateDecay = 1e-2
}

rmsp_config = {
    learningRate = 0.01,
    alpha = 0.99,
    epsilon = 1e-8
}

nag_config = {
    learningRate = 0.01,
    momentum = 0.9
}

adam_config = {
    learningRate = 0.1
}


In [ ]:
sgd_track = compute_track(x_init, f, df, optim.sgd, sgd_config, nsteps)
sgdm_track = compute_track(x_init, f, df, optim.sgd, sgdm_config, nsteps)
nag_track =  compute_track(x_init, f, df, optim.nag, nag_config, nsteps)
rmsp_track = compute_track(x_init, f, df, optim.rmsprop, rmsp_config, nsteps)
adadelta_track = compute_track(x_init, f, df, optim.adadelta, adadelta_config, nsteps)
adagrad_track = compute_track(x_init, f, df, optim.adagrad, adagrad_config, nsteps)

In [ ]:
gnuplotx.figure(2)
gnuplotx.raw(string.format('set xrange [%g:%g]', range[1], range[2]))
gnuplotx.raw(string.format('set yrange [%g:%g]', range[3], range[4]))
gnuplotx.raw(string.format('set zrange [%g:%g]', range[5], range[6]))
gnuplotx.title('Optimizer tracks - sgd/mom/nag')
gnuplotx.displaypaths({xx,yy,zz},{'sgd', sgd_track}, {'sgd+mom', sgdm_track}, {'nag', nag_track})

gnuplotx.figure(3)
gnuplotx.title('Optimizer tracks rmsprop/adagrad/adadelta')
gnuplotx.raw(string.format('set xrange [%g:%g]', range[1], range[2]))
gnuplotx.raw(string.format('set yrange [%g:%g]', range[3], range[4]))
gnuplotx.raw(string.format('set zrange [%g:%g]', range[5], range[6]))
gnuplotx.displaypaths({xx,yy,zz},{'rmsprop', rmsp_track},{'adadelta', adadelta_track},{'adagrad', adagrad_track})